### Univariate Distribution of Features

#### Load and Examine Data

In [ ]:
# Import libraries and modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
# Read in data, examine first few rows
df = pd.read_csv("../data/train.csv")
pd.set_option('display.max_columns', None)
df.head(5)

In [ ]:
print("This data set frame has {} rows and {} columns".format(df.shape[0], df.shape[1]))

In [ ]:
# Check the percent of values in ech column that are missing. 
# We see that there are eight features with 20% or more missing values,
# and two features with over 50% missing values.
# The two column which encode our target (efs and efs_time)
# have no missing values.

pd.DataFrame(df.isna().sum()/df.shape[0] * 100).reset_index() \
    .rename(columns={"index":"Feature", 0:"Percent Missing"}) \
    .sort_values(by="Percent Missing", ascending=False)

In [ ]:
# For now, we will keep all rows and columns--even those with missing data.
# We drop the ID column, which does not contain information useful for modeling.
# We also replace the numerical values in the efs column
# with text values, which are easier to interpret.

df = df.drop("ID", axis=1)
df['efs'] = df['efs'].replace({0:"Censored", 1:"Event"})

In [ ]:
# Check which datatypes exist in the data.
# We have dtype 'O' (for 'object', pandas categorical datatype)
# as well as integer and floating-point datatypes.
# We will confirm that the datatype for each column makes sense
# when we examine the columns individually.
df.dtypes.unique()

In [ ]:
# Compute basic descriptive statistics for numerical variables.
# It appears that many of these features take just a few 
# integer values. 

df_numeric = df.select_dtypes(['float64', 'int64'])
df_numeric.describe()

#### Define Helper Functions to Summarize and Plot Features

In [ ]:
# Function that takes the name of a discrete feature
# and produces a barplot of the number of 
# cases for each value of the feature.
# May be used for either categorical features
# or integer features which only take a
# few distinct values.

def plot_discrete_feature(feat_name, df=df, tick_angle = 0, figsize=(5, 3)):
    fig, ax = plt.subplots(figsize=figsize)
    cat_order = None
    if df[feat_name].dtype == 'O':
        cat_order = df[feat_name].value_counts().index.to_list()
    sns.countplot(df, x = feat_name, order=cat_order)
    plt.xlabel(feat_name)
    plt.ylabel("Number of cases ")
    plt.title("Number of cases by {}".format(feat_name))
    ax.tick_params(axis='x', rotation=tick_angle)
    plt.show()

In [ ]:
# Function the takes the name of a discrete feature
# and returns a dataframe with the percentage
# of cases that take on each level of the feature.
# Note that this ignores any cases where the
# value of the feature is missing.

def get_percentages(feat_name, df=df):
    percentages = df[feat_name].value_counts()/df[feat_name].count() * 100
    formatted = round(pd.DataFrame(percentages).reset_index(), 2) \
         .rename(columns={"count":"percent"}) 
    
    if df[feat_name].dtype == 'O':
        formatted = formatted.sort_values(by="percent", ascending=False)
    else:
        formatted = formatted.sort_values(by = feat_name)
        
    return formatted

In [ ]:
# Function that takes the name of a numeric feature
# and produces a figure with two subplots:
# a boxplot of the feature, and a histogram.
# Boxplots are often useful for detecting outliers,
# while histograms give more insight into the
# overall shape of a distribution.

def plot_numeric_feature(feat_name, data = df, bins=10):
    fig, ax = plt.subplots(1, 2, figsize=(10, 3))
    fig.suptitle("Distribution of {}".format(feat_name))
    sns.boxplot(data = df, y=feat_name, ax=ax[0])
    sns.histplot(data = df, x=feat_name, ax=ax[1], kde=True, bins=bins)
    plt.subplots_adjust(wspace=0.4)
    ax[1].set_ylabel("Number of cases")
    plt.show()

In [ ]:
# While this notebook is mostly for univariate analysis,
# I am including a function here which plots efs_time (survival time)
# broken down by efs (event/censored).
# These two variables can be viewed as jointly encoding
# our true modeling objective (survival time).

def plot_efs_vs_efs_time(bins=10):
    fig, ax = plt.subplots(1, 2, figsize=(12, 3))
    fig.suptitle("Distribution of efs_time by efs")
    sns.boxplot(data = df, y="efs_time", hue="efs", ax=ax[0])
    sns.histplot(data = df, x="efs_time", hue="efs", ax=ax[1], multiple='stack', bins=bins)
    plt.subplots_adjust(wspace=0.3)
    ax[0].legend([], [], frameon=False)
    ax[1].set_ylabel("Number of cases")
    plt.show()

In [ ]:
len(df.columns)

#### Univariate Distributions of Features

##### Distribution of dri_score

In [ ]:
# View percentage of cases by value of dri_score
get_percentages('dri_score')

In [ ]:
# Plot number of cases by value of dri_score
plot_discrete_feature('dri_score', tick_angle=90, figsize=(8, 3))

**Notes:** This feature represents a categorical disease risk index. This is a relatively high cardinality feature, with 11 levels. Of these, 5 levels are relatively rare, representing 5% of less of cases. For simplicity, we may want to combine some levels. For example, `High - TED AML case <missing cytogenetics` could be binned with `High`. Some of these levels seem to represent missing data: `TDB cytogenetics` and `Missing Disease Status` could be coded as missing data.

##### Distribution of psych_disturb

In [ ]:
# View percentage of cases by value of psych_disturb
get_percentages('psych_disturb')

In [ ]:
# Plot number of cases by value of psych_disturb
plot_discrete_feature('psych_disturb')

**Notes:** The value of `psych_disturb` is `Yes` in about 13% of cases where this feature is present. This is relatively rare, but potentially still worth considering. We may want to code the rare value `Not done` as missing data.

##### Distribution of cyto_score

In [ ]:
# View percentage of cases by value of cyto_score
get_percentages('cyto_score')

In [ ]:
# Plot number of cases by value of cyto_score
plot_discrete_feature('cyto_score', figsize=(8, 3))

**Notes:** This categorical variable encodes the level of abnormality observed while analyzing a patient's bone-marror cells. We may wish to code the values `TBD`, `Other` and `Not tested` as missing data. It's possible that `Normal` should be binned with `Favorable`. Analyzing how these levels correlate with survival times help show whether this make sense.

##### Distribution of diabetes

In [ ]:
# View percentage of cases by value of diabetes
get_percentages('diabetes')

In [ ]:
# Plot number of cases by value of diabetes
plot_discrete_feature('diabetes')

**Notes:** For cases where this feature is present, roughly 16% have diabetes. We may wish to thread the `Not done` category as missing data.

##### Distribution of hla_match_c_high

In [ ]:
# View percentage of cases by value of hla_match_c_high
get_percentages('hla_match_c_high')

In [ ]:
# Plot number of cases by value of hla_match_c_high
plot_discrete_feature('hla_match_c_high')

**Notes:** The feature represents recipient / 1st donor level allel level (high resolution) matching a HLA-C. The fact that less than 0.33% of cases have value `0` is striking. It's possible that donors with a poor match on this allele are usually excluded. We may want to examine these specific cases in more detail--they may be outliers, or unusual in some other way.

##### Distribution of hla_high_res_8

In [ ]:
# View percentage of cases by value of hla_high_res_8
get_percentages('hla_high_res_8')

In [ ]:
# Plot number of cases by value of hla_high_res_8
plot_discrete_feature('hla_high_res_8')

**Notes:** This feature represents recipient / 1st donor allele-level (high resolution) matching at multiple genetic loci: HLA-A, HLA-B, HLA-C and HLR-DRB1. This takes discrete integer values up to 8. However, we have essentially no values less than 4. The most common value is 8, at almost 60% of cases.

##### Distribution of tbi_status

In [ ]:
# View percentage of cases by value of tbi_status
get_percentages('tbi_status')

In [ ]:
# Plot number of cases by value of tbi_status
plot_discrete_feature('tbi_status', figsize=(8, 3), tick_angle = 90)

**Notes:** Since we are analyzing Leukemia cases, the meaning of `TBI` in this context is likely ["total-body irradiation"](https://www.cancerresearchuk.org/about-cancer/treatment/bone-marrow-stem-cell-transplants/total-body-irradiation-tbi), which is often performed before a bone-marrow or stem-cell transplant.  We will likely need to bin the feature into fewer levels--for example by grouping all except the first two categories together into a single  `Other` category.

##### Distribution of arrhythmia

In [ ]:
# View percentage of cases by value of arrhythmia
get_percentages('arrhythmia')

In [ ]:
# Plot number of cases by value of arrhythmia
plot_discrete_feature('arrhythmia')

**Notes:** Since the value of arrhythmia is `No` in almost 95% of cases where the feature is present, this feature may not be very useful. Unless it has a very strong relation to the target, this may be one we drop.

##### Distribution of hla_low_res_6

In [ ]:
# View percentage of cases by value of hla_low_res_6
get_percentages('hla_low_res_6')

In [ ]:
# Plot number of cases by value of hla_low_res_6
plot_discrete_feature('hla_low_res_6')

**Notes:** This feature represents ecipient / 1st donor antigen-level (low resolution) matching at HLA-A,-B,-DRB1. Note that this is similar to the definiton of `hla_high_res_8` refers to high-resolution matching and does not include locus HLA-C. Note also that the feature `hla_match_c_high` encodes high-resolution matchong on HLA-C. Hence `hla_high_res_6` may be redundant, unless one or both of the other two features are missing.

As in the case of `hla_high_res_8` and `hla_match_c_high`, this feature is equal to the maximum value a majority of the time when it is present. Values below 3 are very rare and may be outliers.

##### Distribution of graft_type

In [ ]:
# View percentage of cases by value of graft_type
get_percentages('graft_type')

In [ ]:
# Plot number of cases by value of graft_type
plot_discrete_feature('graft_type')

**Notes:** This feature has only two values, and both values are reasonable well-respresented in the data.

##### Distribution of vent_hist

In [ ]:
# View percentage of cases by value of vent_hist
get_percentages('vent_hist')

In [ ]:
# Plot number of cases by value of vent_hist
plot_discrete_feature('vent_hist')

**Notes:** This feature represents history of mechanical ventilation, and is `Yes` less than 3% of the time. Because mechanical ventilation potentially indicates a serious medical issue, this may be worth exploring further. For example, do we see more `Yes` values during the peak COVID-19 years, indicating that it may be a proxy for severe COVID-19 infection?

##### Distribution of renal_issue

In [ ]:
# View percentage of cases by value of renal_issue
get_percentages('renal_issue')

In [ ]:
# Plot number of cases by value of renal_issue
plot_discrete_feature('renal_issue')

**Notes:** This feature encodes the presence of moderate to severe kidney issues, with 98% of the values being `No`. Since this is a low-variance feature, it may not be useful for our model. Alternatively, we may want to combine it with other rare health issues such as history of mechanical ventilation. We may wish to code `Not done` as missing data.

##### Distribution of pulm_severe

In [ ]:
# View percentage of cases by value of pulm_severe
get_percentages('pulm_severe')

In [ ]:
# Plot number of cases by value of pulm_severe
plot_discrete_feature('pulm_severe')

**Notes:** This feature encodes the presence of severe pulmonary issues, with 93% of the values being `No`. Since this is a low-variance feature, it may not be useful for our model. Alternatively, we may want to combine it with other rare health issues such as history of mechanical ventilation. We may wish to code `Not done` as missing data.

##### Distribution of prim_disease_hct

In [ ]:
# View percentage of cases by value of prim_disease_hct
get_percentages('prim_disease_hct')

In [ ]:
# Plot number of cases by value of prim_disease_hct
plot_discrete_feature('prim_disease_hct', figsize=(8, 3), tick_angle=90)

**Notes:** This is a high-cardinality feature. There are 17 values total. Of these, 11 account for less than 5% of cases each. We may wish to bin less-common values together into an "other" category. 

##### Distribution of hla_high_res_6

In [ ]:
# View percentage of cases by value of hla_high_res_6
get_percentages('hla_high_res_6')

In [ ]:
# Plot number of cases by value of hla_high_res_6
plot_discrete_feature('hla_high_res_6')

**Notes:** This feature encodes recipient / 1st donor allele-level (high resolution) matching at HLA-A,-B,-DRB1. This is identical to `hla_low_res_6`, except that the matching is high-resolution instead of low. The distributions of the two features are also quite similar, with a majority of values equal to 6 and virtually none less than 3. These two features are likely to be very highly correlated. However, before discarding either, we may want to examine how often a patient has non-missing data in one of the two, but not both.

##### Distribution of cmv_status

In [ ]:
# View percentage of cases by value of cmv_status
get_percentages('cmv_status')

In [ ]:
# Plot number of cases by value of cmv_status
plot_discrete_feature('cmv_status')

**Notes:**  This feature encodes donor/recipient CMV serostatus--that is, presence of [antibodies to CMV](https://pmc.ncbi.nlm.nih.gov/articles/PMC3512215/) in the donor's and recipient's blood. It appears that a majority of both donors are recipients are positive. However, each of the four posible combinations is represented reasonably well in the data. 

##### Distribution of hla_high_res_10

In [ ]:
# View percentage of cases by value of hla_high_res_10
get_percentages('hla_high_res_10')

In [ ]:
# Plot number of cases by value of hla_high_res_10
plot_discrete_feature('hla_high_res_10')

**Notes:** This feature encodes recipient / 1st donor allele-level (high resolution) matching at generic loci HLA-A,-B,-C,-DRB1, -DQB1. Note that this is identical to `hla_high_res_8` except for present of a new allele, `DQB1`. Since we also have a feature `hla_match_dbq1_high`, the feature may be redunant unless either `hla_high_res_8` or `hla_match_dbq1_high` are missing.

##### Distribution of hla_match_dqb1_high

In [ ]:
# View percentage of cases by value of hla_match_dqb1_high
get_percentages('hla_match_dqb1_high')

In [ ]:
# Plot number of cases by value of hla_match_dqb1_high
plot_discrete_feature('hla_match_dqb1_high')

**Notes:** This feature encodes recipient / 1st donor allele level (high resolution) matching at HLA-DQB1, one of the alleles included in `hla_high_res_10`. As with other HLA match feature we've seen so a majority of cases take the maximum possible value. Virtually no cases have a value 0. These may be outliers.

##### Distribution of tce_imm_match

In [ ]:
# View percentage of cases by value of tce_imm_match
get_percentages('tce_imm_match')

In [ ]:
# Plot number of cases by value of tce_imm_match
plot_discrete_feature('tce_imm_match')

**Notes:** This feature encodes T-cell [epitope](https://en.wikipedia.org/wiki/Epitope) immunogenicity/diversity match. Epiptopes are short sequences of amino acids which are recognized by T-cells. An epiptope mismatch between donor and recipient may cause the recipient's immune system to attack the donor cells. The majority of cases in our data are matches of type P/P. Non-matching combinations between donor and recipient are rare. We may want to bin these as a `mismatched` category.

##### Distribution of hla_nmdp_6

In [ ]:
# View percentage of cases by value of hla_nmdp_6
get_percentages('hla_nmdp_6')

In [ ]:
# Plot number of cases by value of hla_nmdp_6
plot_discrete_feature('hla_nmdp_6')

**Notes:** This feature encodes recipient / 1st donor matching at HLA-A(lo),-B(lo),-DRB1(hi). Assuming "lo" and "hi" refer to low-resolution and high-resolution matching, this should be quite similar to the feature `hla_low_res_6`. However, it's possible that we will have patients with non-missing data in only one of these two features.